In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline

In [4]:
# 我们把解压的数据用下面的代码载入，顺便看看数据的类别分布。
train = pd.read_csv('/Users/wangzhiguo/Downloads/train_modified.csv')
target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol = 'ID'
train['Disbursed'].value_counts() 

0    19680
1      320
Name: Disbursed, dtype: int64

In [5]:
# 选择好样本特征和类别输出
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train['Disbursed']

In [7]:
# http://www.cnblogs.com/pinard/p/6160412.html
# try RF
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
print rf0.oob_score_
y_predprob = rf0.predict_proba(X)[:,1]
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

0.98005
AUC Score (Train): 0.999833


In [ ]:
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(50,201,20)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60, 
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [9]:
rf2 = RandomForestClassifier(n_estimators= 60, max_depth=13, min_samples_split=120,
                                  min_samples_leaf=20,max_features=7 ,oob_score=True, random_state=10)
rf2.fit(X,y)
print rf2.oob_score_

0.984


In [8]:
# # http://www.cnblogs.com/pinard/p/6143927.html
# first try GBDT
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X,y)
y_pred = gbm0.predict(X)
y_predprob = gbm0.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.9852
AUC Score (Train): 0.900531


In [7]:
# find best params
# 首先我们从步长(learning rate)和迭代次数(n_estimators)入手。
# 我们将步长初始值设置为0.1。对于迭代次数进行网格搜索如下：
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_
n_estimators = gsearch1.best_params_['n_estimators']

([mean: 0.81270, std: 0.01969, params: {'n_estimators': 20},
  mean: 0.81343, std: 0.01987, params: {'n_estimators': 30},
  mean: 0.81423, std: 0.01975, params: {'n_estimators': 40},
  mean: 0.81393, std: 0.02079, params: {'n_estimators': 50},
  mean: 0.81546, std: 0.02035, params: {'n_estimators': 60},
  mean: 0.81434, std: 0.01936, params: {'n_estimators': 70},
  mean: 0.81389, std: 0.02073, params: {'n_estimators': 80}],
 {'n_estimators': 60},
 0.81546343051321135)

In [8]:
# find max_depth, min_sample_split
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(
        learning_rate=0.1, n_estimators=n_estimators, min_samples_leaf=20, 
        max_features='sqrt', subsample=0.8, random_state=10), 
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_
max_depth = gsearch2.best_params_['max_depth']
min_samples_split = gsearch2.best_params_['min_samples_split']

([mean: 0.81187, std: 0.02071, params: {'min_samples_split': 100, 'max_depth': 3},
  mean: 0.81271, std: 0.01990, params: {'min_samples_split': 300, 'max_depth': 3},
  mean: 0.81238, std: 0.01937, params: {'min_samples_split': 500, 'max_depth': 3},
  mean: 0.80925, std: 0.02051, params: {'min_samples_split': 700, 'max_depth': 3},
  mean: 0.81846, std: 0.01843, params: {'min_samples_split': 100, 'max_depth': 5},
  mean: 0.81639, std: 0.01800, params: {'min_samples_split': 300, 'max_depth': 5},
  mean: 0.81315, std: 0.01898, params: {'min_samples_split': 500, 'max_depth': 5},
  mean: 0.81262, std: 0.02090, params: {'min_samples_split': 700, 'max_depth': 5},
  mean: 0.81817, std: 0.02014, params: {'min_samples_split': 100, 'max_depth': 7},
  mean: 0.82137, std: 0.01733, params: {'min_samples_split': 300, 'max_depth': 7},
  mean: 0.81703, std: 0.01773, params: {'min_samples_split': 500, 'max_depth': 7},
  mean: 0.81403, std: 0.02354, params: {'min_samples_split': 700, 'max_depth': 7},
  me

In [10]:
param_test3 = {'min_samples_split':range(800,1900,200), 'min_samples_leaf':range(60,101,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(
            learning_rate=0.1, n_estimators=n_estimators, max_depth=max_depth,
            max_features='sqrt', subsample=0.8, random_state=10), 
        param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_
min_samples_split = gsearch3.best_params_['min_samples_split']
min_samples_leaf = gsearch3.best_params_['min_samples_leaf']

In [ ]:
gbm1 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=n_estimators,max_depth=7, min_samples_leaf =60, 
               min_samples_split =1200, max_features='sqrt', subsample=0.8, random_state=10)
gbm1.fit(X,y)
y_pred = gbm1.predict(X)
y_predprob = gbm1.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

In [11]:
max_depth = gsearch2.best_params_['max_depth']
min_samples_split = gsearch2.best_params_['min_samples_split']